# <center> Предсказание победителя в Dota 2
<center> <img src="https://meduza.io/impro/YnJZAHUW6WHz_JQm1uRPkTql_qAhbfxt3oFJLGH7CJg/fill/980/0/ce/1/aHR0cHM6Ly9tZWR1/emEuaW8vaW1hZ2Uv/YXR0YWNobWVudHMv/aW1hZ2VzLzAwNy8x/NTcvNjk1L29yaWdp/bmFsL0tMVThLbUti/ZG5pSzlibDA0Wmlw/WXcuanBn.webp" width="700" height="700">

[Почитать подбробнее](https://meduza.io/feature/2021/10/19/rossiyskaya-komanda-vyigrala-chempionat-mira-po-dota-2-i-poluchila-18-millionov-dollarov-postoyte-otkuda-takie-dengi-neuzheli-igrat-v-dotu-tak-slozhno)

#### [Оригинальная статья](https://arxiv.org/pdf/2106.01782.pdf)
    
### Начало

Посмотрим на готовые признаки и сделаем первую посылку.

1. [Описание данных](#Описание-данных)
2. [Описание признаков](#Описание-признаков)
3. [Наша первая модель](#Наша-первая-модель)
4. [Посылка](#Посылка)

### Первые шаги на пути в датасайенс

5. [Кросс-валидация](#Кросс-валидация)
6. [Что есть в json файлах?](#Что-есть-в-json-файлах?)
7. [Feature engineering](#Feature-engineering)

### Импорты

In [ ]:
import os
import json
import pandas as pd
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

%matplotlib inline

In [ ]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

## <left>Описание данных

Файлы:

- `sample_submission.csv`: пример файла-посылки
- `train_raw_data.jsonl`, `test_raw_data.jsonl`: "сырые" данные
- `train_data.csv`, `test_data.csv`: признаки, созданные авторами
- `train_targets.csv`: результаты тренировочных игр

## <left>Описание признаков
    
Набор простых признаков, описывающих игроков и команды в целом

In [ ]:
PATH_TO_DATA = "/kaggle/input/copy-of-23-24-ml/"

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA,
                                             "train_data.csv"),
                                    index_col="match_id_hash")
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA,
                                            "train_targets.csv"),
                                   index_col="match_id_hash")

In [ ]:
df_train_features.shape

In [ ]:
df_train_features.head()

Имеем ~32 тысячи наблюдений, каждое из которых характеризуется уникальным `match_id_hash` (захэшированное id матча), и 245 признаков. `game_time` показывает момент времени, в который получены эти данные. То есть по сути это не длительность самого матча, а например, его середина, таким образом, в итоге мы сможем получить модель, которая будет предсказывать вероятность победы каждой из команд в течение матча (хорошо подходит для букмекеров).

Нас интересует поле `radiant_win` (так называется одна из команд, вторая - dire). Остальные колоки здесь по сути получены из "будущего" и есть только для тренировочных данных, поэтому на них можно просто посмотреть).

In [ ]:
df_train_targets.head()

## <left>Наша первая модель

In [ ]:
X = df_train_features.values
y = df_train_targets["radiant_win"].values.astype("int8")

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      test_size=0.3,
                                                      random_state=SEED)

#### Обучим случайный лес

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
rf_model.fit(X_train, y_train)

#### Сделаем предсказания и оценим качество на отложенной части данных

In [ ]:
y_pred = rf_model.predict_proba(X_valid)[:, 1]

In [ ]:
valid_score = roc_auc_score(y_valid, y_pred)
print("ROC-AUC score на отложенной части:", valid_score)

Посмотрим на accuracy:

In [ ]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print("Accuracy score (p > 0.5) на отложенной части:", valid_accuracy)

## <left>Посылка

In [ ]:
df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, "test_data.csv"),
                                   index_col="match_id_hash")

X_test = df_test_features.values
y_test_pred = rf_model.predict_proba(X_test)[:, 1]

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred},
                                 index=df_test_features.index)

In [ ]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

## <left>Кросс-валидация

Во многих случаях кросс-валидация оказывается лучше простого разбиения на test и train. Воспользуемся `ShuffleSplit` чтобы создать 5 70%/30% наборов данных.

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc")

In [ ]:
cv_scores_rf

In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

## <left>Что есть в json файлах?

Описание сырых данных можно найти в `train_matches.jsonl` и `test_matches.jsonl`. Каждый файл содержит одну запись для каждого матча в [JSON](https://en.wikipedia.org/wiki/JSON) формате. Его легко превратить в питоновский объект при помощи метода `json.loads`.

In [ ]:
with open(os.path.join(PATH_TO_DATA, "train_raw_data.jsonl")) as fin:
    # прочтем 419 строку
    for i in range(419):
        line = fin.readline()

    # переведем JSON в питоновский словарь
    match = json.loads(line)

In [ ]:
player = match["players"][9]
player["kills"], player["deaths"], player["assists"]

KDA - может быть неплохим признаком, этот показатель считается как:
    
<center>$KDA = \frac{K + A}{D}$

Информация о количестве использованных способностей:

In [ ]:
player["ability_uses"]

In [ ]:
for i, player in enumerate(match["players"]):
    plt.plot(player["times"], player["xp_t"], label=str(i+1))

plt.legend()
plt.xlabel("Time, s")
plt.ylabel("XP")
plt.title("XP change for all players");

#### Сделаем чтение файла с сырыми данными и добавление новых признаков удобным

В этот раз для чтение `json` файлов лучше использовать библиотеку `ujson`, иначе все будет слишком долго :(

In [ ]:
try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ("Подумайте об установке ujson, чтобы работать с JSON объектами быстрее")

try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ("Подумайте об установке tqdm, чтобы следить за прогрессом")


def read_matches(matches_file, total_matches=31698, n_matches_to_read=None):
    """
    Аргуент
    -------
    matches_file: JSON файл с сырыми данными

    Результат
    ---------
    Возвращает записи о каждом матче
    """

    if n_matches_to_read is None:
        n_matches_to_read = total_matches

    c = 0
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            if c >= n_matches_to_read:
                break
            else:
                c += 1
                yield json.loads(line)

#### Чтение данных в цикле

Чтение всех данных занимает 1-2 минуты, поэтому для начала можно попробовать следующее:

1. Читать 10-50 игр
2. Написать код для работы с этими JSON объектами
3. Убедиться, что все работает
4. Запустить код на всем датасете
5. Сохранить результат в `pickle` файл, чтобы в следующий раз не переделывать все заново

## <left>Feature engineering

Напишем функцию, которая поможет нам легче добавлять новые признаки.

In [ ]:
def add_new_features(df_features, matches_file):
    """
    Аргументы
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными

    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    for match in read_matches(matches_file):
        match_id_hash = match['match_id_hash']
        
        # Инициализация переменных для новых признаков
        radiant_tower_kills = dire_tower_kills = radiant_stuns = dire_stuns = 0
        radiant_roshans_killed = dire_roshans_killed = radiant_obs_placed = dire_obs_placed = 0
        radiant_max_level = dire_max_level = 0
        radiant_total_gold = dire_total_gold = radiant_total_xp = dire_total_xp = 0
        radiant_kills = dire_kills = 0

        for player in match["players"]:
            player_team = "radiant" if player["player_slot"] < 128 else "dire"
            
            # Статистика по уровням
            if player_team == "radiant":
                radiant_max_level = max(radiant_max_level, player["level"])
                radiant_obs_placed += player.get("obs_placed", 0)
                radiant_total_gold += player["gold"]
                radiant_total_xp += player["xp"]
                radiant_kills += player["kills"]
            else:
                dire_max_level = max(dire_max_level, player["level"])
                dire_obs_placed += player.get("obs_placed", 0)
                dire_total_gold += player["gold"]
                dire_total_xp += player["xp"]
                dire_kills += player["kills"]
                
            # Статистика по оглушениям и Рошанам
            if player_team == "radiant":
                radiant_stuns += player.get("stuns", 0)
                radiant_roshans_killed += player.get("roshans_killed", 0)
            else:
                dire_stuns += player.get("stuns", 0)
                dire_roshans_killed += player.get("roshans_killed", 0)
                
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                elif objective["team"] == 3:
                    dire_tower_kills += 1

        # Запись в DataFrame
        features_to_add = {
            "radiant_tower_kills": radiant_tower_kills,
            "dire_tower_kills": dire_tower_kills,
            "diff_tower_kills": radiant_tower_kills - dire_tower_kills,
            "radiant_stuns": radiant_stuns,
            "dire_stuns": dire_stuns,
            "diff_stuns": radiant_stuns - dire_stuns,
            "radiant_roshans_killed": radiant_roshans_killed,
            "dire_roshans_killed": dire_roshans_killed,
            "diff_roshans_killed": radiant_roshans_killed - dire_roshans_killed,
            "radiant_max_level": radiant_max_level,
            "dire_max_level": dire_max_level,
            "diff_max_level": radiant_max_level - dire_max_level,
            "radiant_obs_placed": radiant_obs_placed,
            "dire_obs_placed": dire_obs_placed,
            "diff_obs_placed": radiant_obs_placed - dire_obs_placed,
            "radiant_total_gold": radiant_total_gold,
            "dire_total_gold": dire_total_gold,
            "diff_total_gold": radiant_total_gold - dire_total_gold,
            "radiant_total_xp": radiant_total_xp,
            "dire_total_xp": dire_total_xp,
            "diff_total_xp": radiant_total_xp - dire_total_xp,
            "radiant_kills": radiant_kills,
            "dire_kills": dire_kills,
            "diff_kills": radiant_kills - dire_kills
        }

        for feature_name, feature_value in features_to_add.items():
            df_features.loc[match_id_hash, feature_name] = feature_value


In [ ]:
# Скопируем таблицу с признаками
df_train_features_extended = df_train_features.copy()

# Добавим новые
add_new_features(df_train_features_extended,
                 os.path.join(PATH_TO_DATA,
                              "train_raw_data.jsonl"))

In [ ]:
df_train_features_extended.head()

In [ ]:
%%time
cv_scores_base = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
cv_scores_extended = cross_val_score(rf_model, df_train_features_extended.values, y,
                                     cv=cv, scoring="roc_auc", n_jobs=-1)

In [ ]:
print(f"ROC-AUC на кросс-валидации для базовых признаков: {cv_scores_base.mean()}")
print(f"ROC-AUC на кросс-валидации для новых признаков: {cv_scores_extended.mean()}")

In [ ]:
from catboost import CatBoostClassifier

# Подготовка тренировочных данных
X_train_extended = df_train_features_extended.values

# Инициализация и обучение модели CatBoost
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=7,
    verbose=100,
    random_state=SEED
)
cb_model.fit(X_train_extended, y_train, verbose=0)  # verbose=0 для уменьшения выводимой информации
# Предсказания на расширенном тестовом наборе данных
y_test_pred_cb = cb_model.predict_proba(df_test_features_extended.values)[:, 1]
# Создание файла посылки для модели CatBoost
df_submission_cb = pd.DataFrame({
    "match_id_hash": df_test_features_extended.index, 
    "radiant_win_prob": y_test_pred_cb  # Предсказанные вероятности победы команды Radiant
})
submission_filename_cb = "submission.csv" 
df_submission_cb.to_csv(submission_filename_cb, index=False)

Видно, что случайный лес стал работать немного лучше при добавлении новых признаков. A еще нужно, наверное, как-то по-умному закодировать категориальные признаки.

Дальше дело за малым. Добавляйте новые признаки, пробуйте другие методы, которые мы изучили, а также что-то интересное, что мы не прошли. Удачи!